# Project: Metadata Suggesties voor Online Lesmateriaal

##1. Inleiding
In dit ptoject dat we gaan uitvoeren voor de organisatie Wikiwijs is ons doel om metdatasuggesties voor het online lesmateriaal van Wikiwijs te gaan maken. We willen dat ons model uiteindelijk accurate metadata suggesties gaat geven voor het leerjaar, het niveau en het vak.

## 2. Gegevens Onderzoeken en Begrijpen

In deze sectie verkennen we de structuur van onze dataset. We richten ons op JSON-bestanden die lesmateriaal bevatten, specifiek op het bestand vmbo1.json in de submap 'Bio'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import json

# Specificeer het pad naar de submap 'Bio' waar 'vmbo1.json' zich bevindt
directory_path = '/content/drive/My Drive/datalab/Bio'

# Bestandsnaam van het voorbeeld JSON-bestand
file_name = 'vmbo1.json'
file_path = os.path.join(directory_path, file_name)

# Lees het JSON-bestand en print de inhoud om de structuur te begrijpen
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(json.dumps(data, indent=4))  # Print de inhoud met een nette indeling
except Exception as e:
    print(f"Error bij het lezen van bestand {file_name}: {e}")


{
    "documentAuthors": [
        "Pieter Bruring",
        "content VO-",
        "Tom Tahey",
        "Anne Hendriksen",
        "sophia kambakhsh"
    ],
    "uuid": null,
    "authors": [
        {
            "fullName": "Pieter Bruring",
            "referenceName": null,
            "reference": {
                "value": "4"
            },
            "firstName": "Pieter",
            "infix": null,
            "lastName": "Bruring",
            "emailAddress": "p.bruring@kennisnet.org",
            "type": "author"
        },
        {
            "fullName": "content VO-",
            "referenceName": null,
            "reference": {
                "value": "22887"
            },
            "firstName": "content",
            "infix": null,
            "lastName": "VO-",
            "emailAddress": "scala7201@gmail.com",
            "type": "author"
        },
        {
            "fullName": "Tom Tahey",
            "referenceName": null,
            "reference": {
    

**Analyse van vmbo1.json Inhoud**
De JSON-data van vmbo1.json biedt een diepgaand inzicht in het structuur en inhoud van het online lesmateriaal voor het thema "Omgeving" voor vmbo-b12. Hieronder volgt een gedetailleerde analyse van de sleutelcomponenten binnen het bestand:

**Metadata en Auteursinformatie**
Document Auteurs: Het document bevat namen van diverse auteurs, waaronder Pieter Bruring en Sophia Kambakhsh, die bijdragen hebben geleverd aan de inhoud
.

Publicatiedatum: De inhoud is laatst gewijzigd op 25 mei 2023, wat aangeeft dat het materiaal recent is bijgewerkt.


Licentie: Het materiaal valt onder de "CC Naamsvermelding-GelijkDelen 4.0 Internationale licentie", wat betekent dat gebruikers het werk mogen kopiëren, verspreiden, doorgeven, en afgeleide werken mogen maken onder bepaalde voorwaarden.


**Inhoud en Structuur**
Titel: "Thema Omgeving vmbo-b12" geeft duidelijk het onderwerp en het doelpubliek van het lesmateriaal aan.


Beschrijving: Er is een korte inleiding die het thema en de kernonderwerpen beschrijft, zoals levenskenmerken en voedselketens.


Secties: Het document is opgedeeld in verschillende secties, zoals "Intro", "Wat kan ik straks?", en "Wat ga ik doen?", die elk specifieke leerdoelen en activiteiten bevatten. Dit wijst op een gestructureerde benadering om de leerinhoud te presenteren.


**Educatieve Componenten**
Leerdoelen: Voor elke sectie worden specifieke leerdoelen vermeld, zoals het kunnen noemen van negen levenskenmerken en het beschrijven van de begrippen voedselketen en voedselweb.


Opdrachten: Er zijn diverse opdrachten opgenomen met duidelijke instructies en verwachte leeruitkomsten, wat bijdraagt aan een interactieve leerervaring.


Multimedia-inhoud: Het gebruik van afbeeldingen en ingesloten video's verrijkt de tekstuele inhoud en biedt een meer boeiende leerervaring.


**Reflectie en Evaluatie**
Diagnostische toets: Aan het eind van het thema is er een diagnostische toets om de kennis van de leerlingen te evalueren, wat essentieel is voor het beoordelen van het begrip van de leerstof.


Terugkijken: Er is een sectie gewijd aan zelfreflectie, waar leerlingen worden aangemoedigd om na te denken over wat ze hebben geleerd en hoe het proces is verlopen.


**Conclusie**
De inhoud van vmbo1.json toont een goed doordacht en uitgebreid lesmateriaal dat zorgvuldig is ontworpen om vmbo-b12 leerlingen te onderwijzen over het thema "Omgeving". De combinatie van gestructureerde informatie, duidelijke leerdoelen, interactieve opdrachten, en multimedia-inhoud maakt dit een waardevolle bron voor zowel leerlingen als docenten.

##2. Bouw een Input Datapipeline voor Tekst-Preprocessing

We gebruiken BeautifulSoup om HTML-content te verwijderen en reguliere expressies (re) om overbodige spaties te verwijderen. Vervolgens extraheren we de tekst uit elke sectie van onze JSON-bestanden.


In [ ]:
from bs4 import BeautifulSoup
import re

def clean_html(raw_html):
    """Functie om HTML-content te verwerken en overbodige spaties te verwijderen."""
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=' ')
    text = re.sub('\s+', ' ', text).strip()
    return text

def extract_text_from_section(section):
    """Functie om inhoud uit secties te halen, inclusief subsecties."""
    text = section.get('heading', {}).get('content', '') + " "
    for item in section.get('items', []):
        text += item.get('content', '') + " "
    for subsection in section.get('sections', []):
        text += extract_text_from_section(subsection)
    return text


Nu we onze functies voor het schoonmaken en extraheren van tekst hebben, kunnen we deze toepassen op onze dataset. We lezen elk JSON-bestand uit onze dataset, extraheren de tekst met behulp van onze functies, en slaan de schone teksten op voor verdere verwerking.




In [ ]:
import os
import json

# Map waarin alle vakmappen zijn opgeslagen
directory_path = '/content/drive/My Drive/datalab'

# Lijst om alle geëxtraheerde teksten op te slaan
all_texts = []

# Itereren over alle submappen (vakmappen)
for subdir in os.listdir(directory_path):
    subdirectory_path = os.path.join(directory_path, subdir)
    if os.path.isdir(subdirectory_path):
        for filename in os.listdir(subdirectory_path):
            if filename.endswith('.json'):
                file_path = os.path.join(subdirectory_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        data = json.load(file)
                    processed_text = clean_html(extract_text_from_section(data))
                    all_texts.append(processed_text)
                except Exception as e:
                    print(f"Fout bij het verwerken van bestand {filename}: {e}")

# Weergeven van de geëxtraheerde teksten
print(all_texts)


["Thema: Omgeving Intro Biologie betekent leer van het leven . Biologen bestuderen levende wezens (organismen) en de omgeving waarin ze wonen. Het draait dus eigenlijk om de omgeving. In de volgende video ontdek je wat biologie is en wat je allemaal gaat leren bij het vak biologie. Bespreek na het kijken met een klasgenoot wat je denkt te gaan leren bij het vak biologie. In dit eerste thema leer je eerst wat biologen bedoelen met leven. Wat kan ik straks? In de tabel vind je de leerdoelen van dit thema. Aan het einde van dit thema kun je... Opdracht negen levenskenmerken noemen en in eigen woorden vertellen wat deze levenskenmerken inhouden. Levenskenmerken aangeven wanneer iets dood, levend of levenloos is. Levenskenmerken de begrippen voedselketen en voedselweb beschrijven. Voedselweb en voedselketen de begrippen producenten, consumenten en reducenten beschrijven en aangeven welke rol producenten, consumenten en reducenten in een voedselketen spelen. Voedselweb en voedselketen uitleg

Met deze pipeline voor tekst-preprocessing hebben we nu een schone, gestructureerde tekstinput die kan worden gebruikt voor verdere verwerking, zoals feature extractie of direct in een machine learning model. doordat deze functie iterateert over alle submappen zullen de lessen voor elk vak worden meegenomen.

In [ ]:
def compare_texts(raw_html):
    """Functie om de originele en opgeschoonde tekst naast elkaar te tonen."""
    print("Originele Tekst:")
    print(raw_html)
    print("\nOpgeschoonde Tekst:")
    cleaned_text = clean_html(raw_html)
    print(cleaned_text)

# Voorbeeld HTML-tekst
raw_html_example = "<div>Dit is een <b>voorbeeld</b> tekst met <a href='https://example.com'>een link</a> en wat andere <i>HTML</i> elementen.<br />En een nieuwe regel.</div>"

# Vergelijk de originele met de opgeschoonde tekst
compare_texts(raw_html_example)


Originele Tekst:
<div>Dit is een <b>voorbeeld</b> tekst met <a href='https://example.com'>een link</a> en wat andere <i>HTML</i> elementen.<br />En een nieuwe regel.</div>

Opgeschoonde Tekst:
Dit is een voorbeeld tekst met een link en wat andere HTML elementen. En een nieuwe regel.


In [ ]:
import tensorflow as tf

# Omzetten van de lijst met opgeschoonde teksten naar een TensorFlow Dataset
texts_dataset = tf.data.Dataset.from_tensor_slices(all_texts)

# Voorbeeld om de eerste paar elementen van de dataset te bekijken
for text in texts_dataset.take(3):
    print(text.numpy())


b"Thema: Omgeving Intro Biologie betekent leer van het leven . Biologen bestuderen levende wezens (organismen) en de omgeving waarin ze wonen. Het draait dus eigenlijk om de omgeving. In de volgende video ontdek je wat biologie is en wat je allemaal gaat leren bij het vak biologie. Bespreek na het kijken met een klasgenoot wat je denkt te gaan leren bij het vak biologie. In dit eerste thema leer je eerst wat biologen bedoelen met leven. Wat kan ik straks? In de tabel vind je de leerdoelen van dit thema. Aan het einde van dit thema kun je... Opdracht negen levenskenmerken noemen en in eigen woorden vertellen wat deze levenskenmerken inhouden. Levenskenmerken aangeven wanneer iets dood, levend of levenloos is. Levenskenmerken de begrippen voedselketen en voedselweb beschrijven. Voedselweb en voedselketen de begrippen producenten, consumenten en reducenten beschrijven en aangeven welke rol producenten, consumenten en reducenten in een voedselketen spelen. Voedselweb en voedselketen uitleg

In [ ]:
import os
import json
import re

def extract_data_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Initialiseren van containers voor de verzamelde informatie
    leerjaren_set = set()
    niveaus_set = set()
    vakken_set = set()

    for term in data['colophon']['metadata']:
        # Verzamelen van leerjaar informatie
        if term['key'] == 'educationalLevels':
            for value in term['terms'].values():
                # Voeg leerjaren toe aan de set, rekening houdend met meerdere leerjaren
                leerjaren_set.update(re.findall(r'\b\d+\b', value))
                # Voeg niveau informatie toe, waarbij duplicaten worden vermeden
                niveaus_set.add(value.split(',')[0].strip())

        # Verzamelen van vak informatie
        elif term['key'] == 'disciplines':
            vakken_set.update(term['terms'].values())

    # Omzetten van sets naar gesorteerde lijsten en dan naar strings
    leerjaren = ', '.join(sorted(leerjaren_set, key=int))
    niveaus = ', '.join(sorted(niveaus_set))
    vakken = ', '.join(sorted(vakken_set))

    return {'leerjaren': leerjaren, 'niveau': niveaus, 'vak': vakken}

# Pad naar de hoofdmap met submappen die JSON-bestanden bevatten
directory_path = '/content/drive/My Drive/datalab'  # Pas dit aan naar het juiste pad

extracted_data = []

# Doorloop alle submappen en bestanden
for subdir, dirs, files in os.walk(directory_path):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".json"):
            extracted_info = extract_data_from_file(filepath)
            if extracted_info not in extracted_data:
                extracted_data.append(extracted_info)

# Print de verzamelde gegevens
for data in extracted_data:
    print(data)


{'leerjaren': '1, 2', 'niveau': 'VMBO basisberoepsgerichte leerweg', 'vak': 'Biologie, Dynamisch evenwicht, Groei en ontwikkeling, Instandhouding en ontwikkeling, Reproductie, Reproductie en evolutie'}
{'leerjaren': '3, 4', 'niveau': 'VMBO basisberoepsgerichte leerweg, VMBO kaderberoepsgerichte leerweg', 'vak': 'Engels'}


In [ ]:
import os
import json
import re

def extract_data_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    leerjaren_set = set()
    niveaus_set = set()
    vakken_set = set()

    for term in data['colophon']['metadata']:
        if term['key'] == 'educationalLevels':
            for value in\












             term['terms'].values():
                leerjaren_set.update(re.findall(r'\b\d+\b', value))
                niveaus_set.add(value.split(',')[0].strip())

        elif term['key'] == 'disciplines':
            for vak in term['terms'].values():
                # Voeg alleen het eerste woord van elk vak toe aan de set
                vakken_set.add(vak.split()[0])

    leerjaren = ', '.join(sorted(leerjaren_set, key=int))
    niveaus = ', '.join(sorted(niveaus_set))
    # Nu vakken_set bevat alleen het eerste woord, converteren we het naar een string
    vakken = ', '.join(sorted(vakken_set))

    return {'leerjaren': leerjaren, 'niveau': niveaus, 'vak': vakken}

# Pad naar de hoofdmap met submappen die JSON-bestanden bevatten
directory_path = '/content/drive/My Drive/datalab'  # Pas dit aan naar het juiste pad

extracted_data = []

# Doorloop alle submappen en bestanden
for subdir, dirs, files in os.walk(directory_path):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".json"):
            extracted_info = extract_data_from_file(filepath)
            if extracted_info not in extracted_data:
                extracted_data.append(extracted_info)

# Print de verzamelde gegevens
for data in extracted_data:
    print(data)


SyntaxError: invalid syntax (<ipython-input-8-139c5ca46d75>, line 16)

In [ ]:
import os
import json
import re
from bs4 import BeautifulSoup
import tensorflow as tf

class ETLPipeline:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.all_texts = []

    def clean_html(self, raw_html):
        soup = BeautifulSoup(raw_html, "html.parser")
        text = soup.get_text(separator=' ')
        text = re.sub('\s+', ' ', text).strip()
        return text

    def extract_text_from_section(self, section):
        text = section.get('heading', {}).get('content', '') + " "
        for item in section.get('items', []):
            text += item.get('content', '') + " "
        for subsection in section.get('sections', []):
            text += self.extract_text_from_section(subsection)
        return text

    def extract_data_from_file(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        leerjaren_set = set()
        niveaus_set = set()
        vakken_set = set()

        for term in data['colophon']['metadata']:
            if term['key'] == 'educationalLevels':
                for value in term['terms'].values():
                    leerjaren_set.update(re.findall(r'\b\d+\b', value))
                    niveaus_set.add(value.split(',')[0].strip())

            elif term['key'] == 'disciplines':
                vakken_set.update(term['terms'].values())

        leerjaren = ', '.join(sorted(leerjaren_set, key=int))
        niveaus = ', '.join(sorted(niveaus_set))
        vakken = ', '.join(sorted(vakken_set))

        return {'leerjaren': leerjaren, 'niveau': niveaus, 'vak': vakken}

    def extract_text(self):
        for subdir in os.listdir(self.directory_path):
            subdirectory_path = os.path.join(self.directory_path, subdir)
            if os.path.isdir(subdirectory_path):
                for filename in os.listdir(subdirectory_path):
                    if filename.endswith('.json'):
                        file_path = os.path.join(subdirectory_path, filename)
                        try:
                            with open(file_path, 'r', encoding='utf-8') as file:
                                data = json.load(file)
                            processed_text = self.clean_html(self.extract_text_from_section(data))
                            self.all_texts.append(processed_text)
                        except Exception as e:
                            print(f"Error processing file {filename}: {e}")

    def transform_text_to_dataset(self):
        texts_dataset = tf.data.Dataset.from_tensor_slices(self.all_texts)
        return texts_dataset

    def extract_data(self):
        extracted_data = []
        for subdir, dirs, files in os.walk(self.directory_path):
            for filename in files:
                filepath = subdir + os.sep + filename
                if filepath.endswith(".json"):
                    extracted_info = self.extract_data_from_file(filepath)
                    if extracted_info not in extracted_data:
                        extracted_data.append(extracted_info)
        return extracted_data

#how to:
directory_path = '/content/drive/My Drive/datalab'
pipeline = ETLPipeline(directory_path)
pipeline.extract_text()
texts_dataset = pipeline.transform_text_to_dataset()
extracted_data = pipeline.extract_data()

#you can now use texts_dataset and extracted_data as needed